<a href="https://colab.research.google.com/github/chahatpatel2003/CSCI-167/blob/main/notebook_7_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt

def init_params(K, D, sigma_sq_omega):
    np.random.seed(0)
    D_i = 1
    D_o = 1
    all_weights = [None] * (K+1)
    all_biases = [None] * (K+1)
    all_weights[0] = np.random.normal(size=(D, D_i)) * np.sqrt(sigma_sq_omega)
    all_biases[0] = np.zeros((D, 1))
    for layer in range(1, K):
        all_weights[layer] = np.random.normal(size=(D, D)) * np.sqrt(sigma_sq_omega)
        all_biases[layer] = np.zeros((D, 1))
    all_weights[K] = np.random.normal(size=(D_o, D)) * np.sqrt(sigma_sq_omega)
    all_biases[K] = np.zeros((D_o, 1))
    return all_weights, all_biases

def ReLU(preactivation):
    return preactivation.clip(0.0)

def compute_network_output(net_input, all_weights, all_biases):
    K = len(all_weights) - 1
    all_f = [None] * (K+1)
    all_h = [None] * (K+1)
    all_h[0] = net_input
    for layer in range(K):
        all_f[layer] = all_biases[layer] + np.matmul(all_weights[layer], all_h[layer])
        all_h[layer+1] = ReLU(all_f[layer])
    all_f[K] = all_biases[K] + np.matmul(all_weights[K], all_h[K])
    net_output = all_f[K]
    return net_output, all_f, all_h

def run_forward_experiment(K=50, D=80, sigma_list=(1.0, 0.1, 0.01, 2.0/80)):
    n_data = 1000
    data_in = np.random.normal(size=(1, n_data))
    for sigma_sq_omega in sigma_list:
        print(f"\n=== Forward pass with K={K}, D={D}, sigma_sq_omega={sigma_sq_omega:.6f} ===")
        all_weights, all_biases = init_params(K, D, sigma_sq_omega)
        net_output, all_f, all_h = compute_network_output(data_in, all_weights, all_biases)
        stds = []
        for layer in range(1, K+1):
            layer_std = float(np.std(all_h[layer]))
            stds.append(layer_std)
            if layer % 10 == 0 or layer == 1 or layer == K:
                print(f"Layer {layer:2d}, std of hidden units = {layer_std:0.6f}")
        print(f"Summary: first={stds[0]:.6f}, median={np.median(stds):.6f}, last={stds[-1]:.6f}")

run_forward_experiment()

def least_squares_loss(net_output, y):
    return np.sum((net_output - y) * (net_output - y))

def d_loss_d_output(net_output, y):
    return 2*(net_output - y)

def indicator_function(x):
    x_in = np.array(x)
    x_in[x_in >= 0] = 1
    x_in[x_in < 0]  = 0
    return x_in

def backward_pass(all_weights, all_biases, all_f, all_h, y):
    K = len(all_weights) - 1
    all_dl_dweights = [None] * (K+1)
    all_dl_dbiases  = [None] * (K+1)
    all_dl_df       = [None] * (K+1)
    all_dl_dh       = [None] * (K+1)
    all_dl_df[K] = np.array(d_loss_d_output(all_f[K], y))
    for layer in range(K, -1, -1):
        all_dl_dbiases[layer]  = np.array(all_dl_df[layer])
        all_dl_dweights[layer] = np.matmul(all_dl_df[layer], all_h[layer].T)
        all_dl_dh[layer]       = np.matmul(all_weights[layer].T, all_dl_df[layer])
        if layer > 0:
            all_dl_df[layer-1] = indicator_function(all_f[layer-1]) * all_dl_dh[layer]
    return all_dl_dweights, all_dl_dbiases, all_dl_dh, all_dl_df

def run_backward_experiment(K=50, D=80, sigma_list=(1.0, 0.1, 0.01, 2.0/80)):
    n_data = 100
    for sigma_sq_omega in sigma_list:
        print(f"\n=== Backward pass with K={K}, D={D}, sigma_sq_omega={sigma_sq_omega:.6f} ===")
        all_weights, all_biases = init_params(K, D, sigma_sq_omega)
        aggregate_dl_df = [None] * (K+1)
        for layer in range(1, K):
            aggregate_dl_df[layer] = np.zeros((D, n_data))
        for c_data in range(n_data):
            data_in = np.random.normal(size=(1,1))
            y = np.zeros((1,1))
            net_output, all_f, all_h = compute_network_output(data_in, all_weights, all_biases)
            all_dl_dweights, all_dl_dbiases, all_dl_dh, all_dl_df = backward_pass(all_weights, all_biases, all_f, all_h, y)
            for layer in range(1, K):
                aggregate_dl_df[layer][:, c_data] = np.squeeze(all_dl_df[layer])
        stds = []
        for layer in range(1, K):
            s = float(np.std(aggregate_dl_df[layer].ravel()))
            stds.append(s)
            if layer % 10 == 0 or layer == 1 or layer == K-1:
                print(f"Layer {layer:2d}, std of dl_df = {s:0.6f}")
        print(f"Summary: first={stds[0]:.6f}, median={np.median(stds):.6f}, last={stds[-1]:.6f}")

run_backward_experiment()



=== Forward pass with K=50, D=80, sigma_sq_omega=1.000000 ===
Layer  1, std of hidden units = 0.656872
Layer 10, std of hidden units = 8984991.951712
Layer 20, std of hidden units = 571669882459492.125000
Layer 30, std of hidden units = 123600525236285774757888.000000
Layer 40, std of hidden units = 3871517592822056449697747304448.000000
Layer 50, std of hidden units = 263828723966224442553385425202470453248.000000
Summary: first=0.656872, median=37329899293413752832.000000, last=263828723966224442553385425202470453248.000000

=== Forward pass with K=50, D=80, sigma_sq_omega=0.100000 ===
Layer  1, std of hidden units = 0.207721
Layer 10, std of hidden units = 89.849920
Layer 20, std of hidden units = 57166.988246
Layer 30, std of hidden units = 123600525.236286
Layer 40, std of hidden units = 38715175928.220612
Layer 50, std of hidden units = 26382872396622.488281
Summary: first=0.207721, median=4702168.299271, last=26382872396622.488281

=== Forward pass with K=50, D=80, sigma_sq_ome